In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
pre = pd.read_csv('anon_pre.csv')
post = pd.read_csv('anon_post.csv')
cis = pd.read_csv('anon_cis.csv')
# matched = pre.set_index('anon_student_id').join(post.set_index('anon_student_id'), rsuffix='_post').dropna(subset=['q01a', 'q01a_post']).reset_index()
pre.set_index(['anon_student_id'])

In [3]:
print('----Overview----')
print('Total number of instances of the survey everywhere:', cis.shape[0])
print('Number of unique universities in data set:', cis.anon_university_id.nunique())

print('Number of unique instructors in data set:', cis.anon_instructor_id.nunique())

# this works becuase students were given anonymous IDs sequentially starting from 0
print('Number of unique students in data set (unmatched):', post.anon_student_id.max())

# this is checking if the students answered at least the first question of both pre and post and then calling that matched.
print('Number of unique students in data set (matched):', matched.index.nunique())

print('Number of instructors with repeated surveys:', cis.groupby('anon_instructor_id').count().groupby('Q5').count().Q52.iloc[1:].sum())

print('Number of universities with repeated surveys:', cis.groupby('anon_university_id').count().groupby('Q5').count().Q52.iloc[1:].sum())

print('\n')
pre_completion = pre.groupby('survey_id').count().anon_student_id.reset_index()
pre_completion.columns = ['pre_survey_id', 'pre_cnt']
post_completion = post.groupby('survey_id').count().anon_student_id.reset_index()
post_completion.columns = ['post_survey_id', 'post_cnt']
cisq19 = cis[['Q19', 'pre_survey_id', 'post_survey_id']]
pre19 = cisq19.merge(pre_completion)
post19 = cisq19.merge(post_completion)

matched_cnt = pre.set_index('anon_student_id').join(post.set_index('anon_student_id'), rsuffix='_post').dropna(subset=['q01a', 'q01a_post']).groupby('survey_id').count().duration.reset_index()
matched_cnt.columns = ['pre_survey_id', 'cnt']
matched_cnt = cis.merge(matched_cnt)[['pre_survey_id', 'Q19', 'cnt']]


print('Number of surveys with more student respondents than reported maximum registered students (PRE):', (pre19.pre_cnt/pre19.Q19).apply(lambda x: x if x > 1 else np.nan).dropna().shape[0])
print('Number of surveys with more student respondents than reported maximum registered students (POST):', (post19.post_cnt/post19.Q19).apply(lambda x: x if x > 1 else np.nan).dropna().shape[0])

print('Number of surveys with more student respodnents than repored maximum registered students (MATCHED):', (matched_cnt.cnt/matched_cnt.Q19).apply(lambda x: x if x > 1 else np.nan).dropna().shape[0])

print('Average fraction +\- STDEV of course completing the PRE survey:', round((pre19.pre_cnt/pre19.Q19).apply(lambda x: x if x <= 1 else np.nan).dropna().mean(), 2), '+/-', round((pre19.pre_cnt/pre19.Q19).apply(lambda x: x if x <= 1 else np.nan).dropna().std(), 2)
)
print('Average +\- STDEV fraction of course completing the POST survey:', round((post19.post_cnt/post19.Q19).apply(lambda x: x if x <= 1 else np.nan).dropna().mean(), 2), '+/-', round((post19.post_cnt/post19.Q19).apply(lambda x: x if x <= 1 else np.nan).dropna().std(), 2))
print('Average +\- STDEV fraction of course completing the MATCHED survey:', round((matched_cnt.cnt/matched_cnt.Q19).apply(lambda x: x if x <= 1 else np.nan).dropna().mean(), 2), '+/-', round((matched_cnt.cnt/matched_cnt.Q19).apply(lambda x: x if x <= 1 else np.nan).dropna().std(), 2))

print('\n')

print('----Gender----')
# unmatched_gender = post.groupby('Q54').count().anon_student_id
unmatched_gender = post[['anon_student_id', 'Q54']].drop_duplicates().groupby('Q54').count().anon_student_id
print('Number of females in data set (unmatched):', unmatched_gender[1])
print('Number of males in data set (unmatched):', unmatched_gender[2])
print('Number of other in data set (unmatched):', unmatched_gender[3])
print('\n')

# matched_gender = matched.groupby('Q54').count().anon_student_id
matched_gender = matched[['anon_student_id', 'Q54']].drop_duplicates().groupby('Q54').count().anon_student_id
print('Number of females in data set (matched):', matched_gender[1])
print('Number of males in data set (matched):', matched_gender[2])
print('Number of other in data set (matched):', matched_gender[3])
print('\n')


print('----Race (unmatched)----')
unmatched_race = post[['anon_student_id', 'Q52_1', 'Q52_2', 'Q52_3', 'Q52_4', 'Q52_5', 'Q52_6', 'Q52_7']].drop_duplicates()
print('American Indian or Alaska Native:', unmatched_race.Q52_1.sum())
print('Asian:', unmatched_race.Q52_2.sum())
print('Black or African American:', unmatched_race.Q52_3.sum())
print('Hispanic/Latino:', unmatched_race.Q52_4.sum())
print('Native Hawaiian or other Pacific Islander:', unmatched_race.Q52_5.sum())
print('White:', unmatched_race.Q52_6.sum())
print('Other race/ethnicity:', unmatched_race.Q52_7.sum())
s = unmatched_race[['Q52_1', 'Q52_2', 'Q52_3', 'Q52_4', 'Q52_5', 'Q52_6', 'Q52_7']].sum(axis=1)
print('More than one race:', s.where(s > 1).count())
print('\n')

print('----Race (matched)----')
matched_race = matched[['anon_student_id', 'Q52_1', 'Q52_2', 'Q52_3', 'Q52_4', 'Q52_5', 'Q52_6', 'Q52_7']].drop_duplicates()
print('American Indian or Alaska Native:', matched_race.Q52_1.sum())
print('Asian:', matched_race.Q52_2.sum())
print('Black or African American:', matched_race.Q52_3.sum())
print('Hispanic/Latino:', matched_race.Q52_4.sum())
print('Native Hawaiian or other Pacific Islander:', matched_race.Q52_5.sum())
print('White:', matched_race.Q52_6.sum())
print('Other race/ethnicity:', matched_race.Q52_7.sum())
s = matched_race[['Q52_1', 'Q52_2', 'Q52_3', 'Q52_4', 'Q52_5', 'Q52_6', 'Q52_7']].sum(axis=1)
print('More than one race:', s.where(s > 1).count())
print('\n')

print('----Course information----')
course_type = cis.groupby('Q18')

print('Number of times survey given to students in first year lab:', course_type.count().Q5.loc['First year (introductory) lab'])
print('Number of times survey given to students in BUFFY lab courses:', course_type.count().Q5.loc['Beyond the first year lab'])

## haha, i guess its not a good assumption to think its normally distributed course enrollment numbers
print('Average +/- STDEV students in first year lab:', course_type.mean().Q19.loc['First year (introductory) lab'].round(2), '+/-',course_type.std().Q19.loc['First year (introductory) lab'].round(2))
print('Average +/- STDEV students in BUFFY lab courses:', course_type.mean().Q19.loc['Beyond the first year lab'].round(2), '+/-', course_type.std().Q19.loc['Beyond the first year lab'].round(2))

course_type = cis.groupby('Q27')
print('Algebra based intro courses:', course_type.count().Q5.loc['Algebra-based'])
print('Calculus based intro courses:', course_type.count().Q5.loc['Calculus-based'])

----Overview----
Total number of instances of the survey everywhere: 599
Number of unique universities in data set: 133
Number of unique instructors in data set: 204
Number of unique students in data set (unmatched): 43080
Number of unique students in data set (matched): 30534
Number of instructors with repeated surveys: 100
Number of universities with repeated surveys: 80


Number of surveys with more student respondents than reported maximum registered students (PRE): 32
Number of surveys with more student respondents than reported maximum registered students (POST): 17
Number of surveys with more student respodnents than repored maximum registered students (MATCHED): 51
Average fraction +\- STDEV of course completing the PRE survey: 0.71 +/- 0.25
Average +\- STDEV fraction of course completing the POST survey: 0.58 +/- 0.25
Average +\- STDEV fraction of course completing the MATCHED survey: 0.49 +/- 0.27


----Gender----
Number of females in data set (unmatched): 11620
Number of mal

In [4]:
matched_cnt = pre.set_index('anon_student_id').join(post.set_index('anon_student_id'), rsuffix='_post').dropna(subset=['q01a', 'q01a_post']).groupby('survey_id').count().duration.reset_index()
matched_cnt.columns = ['pre_survey_id', 'cnt']
matched_cnt = cis.merge(matched_cnt)[['pre_survey_id', 'Q19', 'cnt']]
round((matched_cnt.cnt/matched_cnt.Q19).apply(lambda x: x if x <= 1 else np.nan).dropna().mean(), 2)

0.49

In [5]:
pre_completion = pre.groupby('survey_id').count().anon_student_id.reset_index()
pre_completion.columns = ['pre_survey_id', 'pre_cnt']
post_completion = post.groupby('survey_id').count().anon_student_id.reset_index()
post_completion.columns = ['post_survey_id', 'post_cnt']
cisq19 = cis[['Q19', 'pre_survey_id', 'post_survey_id']]
# cisq19.set_index('pre_survey_id').join(pre_completion.set_index('post_survey_id'))
# cisq19.merge(pre_completion).merge(post_completion)
pre19 = cisq19.merge(pre_completion)
# (pre19.pre_cnt/pre19.Q19).apply(lambda x: x if x ==2 else np.nan).dropna()
(pre19.pre_cnt/pre19.Q19).apply(lambda x: x if x <= 1 else np.nan).dropna().std()

0.24913458710907008

In [17]:
matched.groupby('anon_student_id').count().sort_values('survey_id')

,survey_id,duration,q01a,q01b,q27a,q27b,q03a,q03b,q22a,q22b,...,q28c,q29a_post,q29b_post,q30a_post,q30b_post,q30c,q31a_post,q31b_post,q40a_post,q40b_post
anon_student_id,,,,,,,,,,,,,,,,,,,,,
5,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
22638,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
22637,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
22635,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
22634,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
995,12,12,12,12,12,12,12,12,12,12,...,12,12,12,12,12,12,12,12,12,12
1624,12,12,12,12,12,12,12,12,12,12,...,12,12,12,12,12,12,12,12,12,12


In [20]:
matched[matched.anon_student_id==1950]

,anon_student_id,survey_id,duration,q01a,q01b,q27a,q27b,q03a,q03b,q22a,...,q28c,q29a_post,q29b_post,q30a_post,q30b_post,q30c,q31a_post,q31b_post,q40a_post,q40b_post
5639,1950,56LgIcKYYypU3K5,410.0,5.0,5.0,5.0,5.0,3.0,5.0,5.0,...,4.0,3.0,5.0,3.0,5.0,4.0,5.0,5.0,4.0,4.0
5640,1950,56LgIcKYYypU3K5,410.0,5.0,5.0,5.0,5.0,3.0,5.0,5.0,...,3.0,3.0,5.0,3.0,3.0,4.0,5.0,5.0,4.0,4.0
5641,1950,56LgIcKYYypU3K5,410.0,5.0,5.0,5.0,5.0,3.0,5.0,5.0,...,3.0,3.0,5.0,3.0,5.0,3.0,5.0,5.0,4.0,4.0
5642,1950,56LgIcKYYypU3K5,410.0,5.0,5.0,5.0,5.0,3.0,5.0,5.0,...,3.0,5.0,5.0,1.0,3.0,3.0,5.0,5.0,4.0,4.0
5643,1950,3scq3Lugl5QdmER,315.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,4.0,3.0,5.0,3.0,5.0,4.0,5.0,5.0,4.0,4.0
5644,1950,3scq3Lugl5QdmER,315.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,3.0,3.0,5.0,3.0,3.0,4.0,5.0,5.0,4.0,4.0
5645,1950,3scq3Lugl5QdmER,315.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,3.0,3.0,5.0,3.0,5.0,3.0,5.0,5.0,4.0,4.0
5646,1950,3scq3Lugl5QdmER,315.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,3.0,5.0,5.0,1.0,3.0,3.0,5.0,5.0,4.0,4.0
5647,1950,4TuvFyMCVolV6Vn,227.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,4.0,3.0,5.0,3.0,5.0,4.0,5.0,5.0,4.0,4.0
5648,1950,4TuvFyMCVolV6Vn,227.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,3.0,3.0,5.0,3.0,3.0,4.0,5.0,5.0,4.0,4.0


In [23]:
cis[cis.pre_survey_id=='3scq3Lugl5QdmER']

,Q5,Q52,Q53,Q18,Q27,Q6,Q11,Q19,Q20,Q15,...,Q38_4,Q41,Q42,Q43,StartDate,anon_instructor_id,anon_university_id,ResponseId,pre_survey_id,post_survey_id
540,quarter,NaN,Fall,First year (introductory) lab,Calculus-based,10/01/2018,Credit for completion (like an assignment),413,18,NaN,...,NaN,NaN,NaN,NaN,09/19/2018 13:24,189,121,R_3QDaRh0QYfU9TxK,3scq3Lugl5QdmER,01lNTCoyXO8Z1rf


In [8]:
pre[pre.survey_id=='2iDeu0O3EPqxT4V'].anon_student_id

1355    1350
1356    1351
1357    1352
1358    1353
1359    1354
        ... 
1413    1408
1414    1409
1415    1410
1416    1411
1417    1412
Name: anon_student_id, Length: 63, dtype: int64

In [9]:
cis[cis.pre_survey_id=='2iDeu0O3EPqxT4V']

,Q5,Q52,Q53,Q18,Q27,Q6,Q11,Q19,Q20,Q15,...,Q38_4,Q41,Q42,Q43,StartDate,anon_instructor_id,anon_university_id,ResponseId,pre_survey_id,post_survey_id
215,quarter,NaN,Winter,First year (introductory) lab,Algebra-based,02/24/2019,No incentive,24,6,Master's granting institution,...,Sometimes,7.0,4.0,1.0,02/12/2019 06:54,96,54,R_0GPGaYXN7Pxw8SZ,2iDeu0O3EPqxT4V,d54xJUggRYuvorP


Index(['anon_student_id', 'survey_id', 'duration', 'q01a', 'q01b', 'q27a',
       'q27b', 'q03a', 'q03b', 'q22a', 'q22b', 'q06a', 'q06b', 'q19a', 'q19b',
       'q24a', 'q24b', 'q10a', 'q10b', 'q28a', 'q28b', 'q05a', 'q05b', 'q11a',
       'q11b', 'q02a', 'q02b', 'q26a', 'q26b', 'q13a', 'q13b', 'q15a', 'q15b',
       'q30a', 'q30b', 'q16a', 'q16b', 'q18a', 'q18b', 'q20a', 'q20b', 'q23a',
       'q23b', 'q09a', 'q09b', 'q21a', 'q21b', 'q40a', 'q40b', 'q12a', 'q12b',
       'q25a', 'q25b', 'q17a', 'q17b', 'q04a', 'q04b', 'q14a', 'q14b', 'q29a',
       'q29b', 'q07a', 'q07b', 'q31a', 'q31b'],
      dtype='object')

In [10]:
pre19.loc[268]

Q19                            17
pre_survey_id     6FQgPIcTNGjmkSx
post_survey_id    b48qtxCi3S7Uwuh
pre_cnt                        17
Name: 268, dtype: object

In [11]:
cis[cis.pre_survey_id=='6FQgPIcTNGjmkSx']

,Q5,Q52,Q53,Q18,Q27,Q6,Q11,Q19,Q20,Q15,...,Q38_4,Q41,Q42,Q43,StartDate,anon_instructor_id,anon_university_id,ResponseId,pre_survey_id,post_survey_id
327,semester,Spring,NaN,Beyond the first year lab,NaN,01/25/2017,Credit for completion (like an assignment),17,1,PhD granting institution,...,Never,1.0,2.0,0.0,01/16/2017 10:43,124,70,R_30ohCR7G2gMkg7a,6FQgPIcTNGjmkSx,b48qtxCi3S7Uwuh


In [12]:
pre[pre.survey_id=='6FQgPIcTNGjmkSx']

,anon_student_id,survey_id,duration,q01a,q01b,q27a,q27b,q03a,q03b,q22a,...,q04a,q04b,q14a,q14b,q29a,q29b,q07a,q07b,q31a,q31b
38734,34648,6FQgPIcTNGjmkSx,865.0,5.0,5.0,5.0,5.0,1.0,1.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
38735,34649,6FQgPIcTNGjmkSx,571.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
38736,5385,6FQgPIcTNGjmkSx,310.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,...,5.0,5.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,5.0
38737,34650,6FQgPIcTNGjmkSx,180.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
38738,34651,6FQgPIcTNGjmkSx,137.0,5.0,5.0,5.0,5.0,5.0,5.0,3.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
38739,34652,6FQgPIcTNGjmkSx,1486.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
38740,34653,6FQgPIcTNGjmkSx,281.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,1.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
38741,34654,6FQgPIcTNGjmkSx,355524.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,3.0,3.0
38742,34655,6FQgPIcTNGjmkSx,6430.0,5.0,5.0,3.0,5.0,3.0,5.0,1.0,...,5.0,5.0,1.0,1.0,3.0,5.0,3.0,5.0,5.0,5.0
38743,34656,6FQgPIcTNGjmkSx,1753.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0


In [13]:
pre.shape

(40072, 65)

In [14]:
pre.drop_duplicates()

,anon_student_id,survey_id,duration,q01a,q01b,q27a,q27b,q03a,q03b,q22a,...,q04a,q04b,q14a,q14b,q29a,q29b,q07a,q07b,q31a,q31b
0,0,0NBvjWEubiZYgcJ,381.0,5.0,5.0,1.0,5.0,1.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,5.0
1,1,0NBvjWEubiZYgcJ,497.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,1.0,5.0,3.0,5.0,5.0,5.0
2,2,0NBvjWEubiZYgcJ,4382.0,5.0,5.0,5.0,5.0,3.0,5.0,3.0,...,5.0,5.0,3.0,5.0,5.0,5.0,1.0,5.0,5.0,5.0
3,3,0NBvjWEubiZYgcJ,362.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,...,5.0,5.0,5.0,5.0,3.0,5.0,5.0,5.0,5.0,5.0
4,4,0NBvjWEubiZYgcJ,706.0,5.0,5.0,1.0,5.0,5.0,5.0,1.0,...,1.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40067,35761,dpp1pzjQkszbz6Z,260.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,...,5.0,5.0,5.0,5.0,5.0,5.0,1.0,1.0,5.0,5.0
40068,35762,dpp1pzjQkszbz6Z,534.0,5.0,5.0,5.0,5.0,5.0,5.0,3.0,...,5.0,5.0,3.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
40069,35763,dpp1pzjQkszbz6Z,507.0,5.0,1.0,3.0,1.0,1.0,3.0,1.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
40070,35764,dpp1pzjQkszbz6Z,147.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,...,5.0,5.0,5.0,5.0,5.0,5.0,1.0,1.0,5.0,5.0


In [15]:
post.shape

(31592, 110)

In [16]:
post.drop_duplicates()

,anon_student_id,survey_id,duration,Q47,Q48,Q49,Q50,Q52_1,Q52_2,Q52_3,...,q28c,q29a,q29b,q30a,q30b,q30c,q31a,q31b,q40a,q40b
0,28,becDjZTnwKwrCpT,613.0,5.0,NaN,4.0,1.0,NaN,NaN,NaN,...,4.0,5.0,5.0,5.0,5.0,3.0,5.0,5.0,4.0,4.0
1,30,becDjZTnwKwrCpT,867.0,5.0,10.0,3.0,2.0,NaN,1.0,NaN,...,4.0,3.0,5.0,1.0,1.0,4.0,5.0,5.0,4.0,4.0
2,31,becDjZTnwKwrCpT,635.0,11.0,NaN,4.0,1.0,NaN,NaN,NaN,...,4.0,5.0,5.0,1.0,5.0,5.0,5.0,5.0,4.0,4.0
3,39,becDjZTnwKwrCpT,599.0,5.0,NaN,4.0,2.0,NaN,NaN,NaN,...,5.0,5.0,5.0,5.0,5.0,4.0,5.0,5.0,4.0,4.0
4,23,becDjZTnwKwrCpT,690.0,11.0,NaN,3.0,1.0,NaN,NaN,NaN,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31587,43078,8ijnZC31JG1q7Mp,271.0,5.0,NaN,3.0,3.0,NaN,NaN,1.0,...,3.0,3.0,3.0,1.0,1.0,3.0,3.0,3.0,4.0,4.0
31588,35737,8ijnZC31JG1q7Mp,625.0,5.0,11.0,2.0,2.0,NaN,NaN,1.0,...,5.0,5.0,5.0,1.0,1.0,5.0,5.0,5.0,4.0,4.0
31589,35744,8ijnZC31JG1q7Mp,1415.0,5.0,NaN,3.0,1.0,NaN,NaN,1.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0
31590,43079,8ijnZC31JG1q7Mp,442.0,5.0,NaN,4.0,1.0,NaN,NaN,NaN,...,4.0,5.0,5.0,3.0,3.0,4.0,5.0,5.0,4.0,4.0
